<a href="https://colab.research.google.com/github/Aryanmashru/LTX-2/blob/main/SadTalker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title **setup（about 5 minutes）**
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!python --version
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/cedro3/SadTalker.git &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg &> /dev/null
!python3.8 -m pip install -r requirements.txt


update-alternatives: error: alternative path /usr/bin/python3.8 doesn't exist
update-alternatives: error: alternative path /usr/bin/python3.9 doesn't exist
Python 3.12.12
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,361 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illin

In [2]:
#@title **download model（about 1 minute)**
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

Download pre-trained models...
--2026-02-11 06:21:44--  https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/569518584/ccc415aa-c6f4-47ee-8250-b10bf440ba62?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-11T07%3A00%3A13Z&rscd=attachment%3B+filename%3Dmapping_00109-model.pth.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-02-11T05%3A59%3A52Z&ske=2026-02-11T07%3A00%3A13Z&sks=b&skv=2018-11-09&sig=6jv7zM2ZUZwR8ta5P%2BlR7Qaidirk3dc73lyyJl9CII8%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc3MDc5NDUwNCwib

In [3]:
#@title **inference for face**
image ='full3.png' #@param {type:"string"}
audio ='eluosi.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python3.8 inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --enhancer gfpgan

/bin/bash: line 1: python3.8: command not found


In [4]:
#@title **play movie**
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

IndexError: list index out of range

In [11]:
!sed -n '1,40p' src/face3d/util/preprocess.py
!sed -i 's/warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)/warnings.filterwarnings("ignore")/g' src/face3d/util/preprocess.py


"""This script contains the image preprocessing code for Deep3DFaceRecon_pytorch
"""

import numpy as np
from scipy.io import loadmat
from PIL import Image
import cv2
import os
from skimage import transform as trans
import torch
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 


# calculating least square problem for image alignment
def POS(xp, x):
    npts = xp.shape[1]

    A = np.zeros([2*npts, 8])

    A[0:2*npts-1:2, 0:3] = x.transpose()
    A[0:2*npts-1:2, 3] = 1

    A[1:2*npts:2, 4:7] = x.transpose()
    A[1:2*npts:2, 7] = 1

    b = np.reshape(xp.transpose(), [2*npts, 1])

    k, _, _, _ = np.linalg.lstsq(A, b)

    R1 = k[0:3]
    R2 = k[4:7]
    sTx = k[3]
    sTy = k[7]
    s = (np.linalg.norm(R1) + np.linalg.norm(R2))/2
    t = np.stack([sTx, sTy], axis=0)

    return t, s
    


In [17]:
!pip install kornia
!pip install facexlib
!pip install yacs


In [26]:
content = '''enhancer_list = ["none"]

def enhancer_generator_with_len(*args, **kwargs):
    return None, 0
'''
with open("src/utils/face_enhancer.py", "w") as f:
    f.write(content)

print("face_enhancer.py patched successfully")


face_enhancer.py patched successfully


In [29]:
!ls -lh checkpoints


total 1.7G
-rw-r--r-- 1 root root 149M May 31  2023 mapping_00109-model.pth.tar
-rw-r--r-- 1 root root 692M May 31  2023 mapping_00109-model.safetensors
-rw-r--r-- 1 root root 149M May 31  2023 mapping_00229-model.pth.tar
-rw-r--r-- 1 root root 692M May 31  2023 SadTalker_V0.0.2_256.safetensors


In [31]:
import site
import os

# Find facexlib alignment file
for p in site.getsitepackages():
    target = os.path.join(p, "facexlib/alignment/awing_arch.py")
    if os.path.exists(target):
        file_path = target
        break

print("Patching:", file_path)

with open(file_path, "r") as f:
    code = f.read()

code = code.replace("np.float", "float")

with open(file_path, "w") as f:
    f.write(code)

print("facexlib patched successfully")


Patching: /usr/local/lib/python3.12/dist-packages/facexlib/alignment/awing_arch.py
facexlib patched successfully


In [33]:
import site, os

for p in site.getsitepackages():
    path = os.path.join(p, "../SadTalker/src/face3d/util/preprocess.py")
    if os.path.exists(path):
        file_path = os.path.abspath(path)
        break

print("Patching:", file_path)

with open(file_path, "r") as f:
    code = f.read()

# Fix translation + disable zoom scaling
code = code.replace(
    "trans_params = np.array([w0, h0, s, t[0], t[1]])",
    "trans_params = np.array([float(w0), float(h0), 1.0, float(t[0]), float(t[1])])"
)

with open(file_path, "w") as f:
    f.write(code)

print("align_img patched (zoom disabled)")


Patching: /usr/local/lib/python3.12/dist-packages/facexlib/alignment/awing_arch.py
align_img patched (zoom disabled)


In [38]:
#@title **inference for portrait**
image ='ref.png' #@param {type:"string"}
audio ='tts.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python inference.py \
--driven_audio $driven_audio \
--source_image $source_image \
--result_dir ./results \
--still \
--preprocess full \
--size 256 \
--enhancer none \
--verbose


using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00,  9.55it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 20.02it/s]
mel:: 100% 174/174 [00:00<00:00, 47828.09it/s]
audio2exp:: 100% 18/18 [00:00<00:00, 244.27it/s]
Face Renderer:: 100% 87/87 [01:06<00:00,  1.32it/s]
The generated video is named ./results/2026_02_11_06.50.01/ref##tts.mp4
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
seamlessClone:: 100% 174/174 [00:06<00:00, 28.57it/s]
The generated video is named ./results/2026_02_11_06.50.01/ref##tts_full.mp4
Traceback (most recent call last):
  File "/content/SadTalker/src/facerender/animate.py", line 244, in generate
    imageio.mimsave(enhanced_path, enhanced_images_gen_with_len, fps=float(25))
  File "/usr/local/lib/python3.12/dist-packages/imageio/v2.py", line 495, in mimwrite
    return file.write(ims, is_batc

In [36]:
!ls -lh results


total 20K
drwxr-xr-x 2 root root 4.0K Feb 11 06:38 2026_02_11_06.38.15
drwxr-xr-x 3 root root 4.0K Feb 11 06:40 2026_02_11_06.39.57
drwxr-xr-x 3 root root 4.0K Feb 11 06:42 2026_02_11_06.42.10
drwxr-xr-x 3 root root 4.0K Feb 11 06:43 2026_02_11_06.43.14
drwxr-xr-x 3 root root 4.0K Feb 11 06:46 2026_02_11_06.45.01


In [37]:
#@title **play movie**
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

IndexError: list index out of range